# 3조 News 데이터 활용

# SBS

In [1]:
def get_sbs():
    
    base_url = 'https://news.sbs.co.kr/news/newsSection.do'

    params = {
        'sectionType' : '01',
        'plink' : 'SNB',
        'cooper' : 'SBSNEWS',
        'pageIdx' : 1,
        'pageDate' : 20200806
    }

    index_list = ['01', '02', '03', '07', '08', '14', '09'] # SectionType ( 카테고리를 가져오기 위한 리스트)

    index_str_list = ['정치', '경제', '사회', '국제', '생활&문화', '연예', '스포츠'] # SectionType = 카테고리 이름

    index_point = 0 # 리스트를 가져 오기 위한 index

    txt_file_name = ['a1.txt', 'b1.txt', 'c1.txt', 'd1.txt', 'e1.txt', 'f1.txt', 'g1.txt'] # 각 카테고리별 저장을 위한 text 파일 list


    f = open(txt_file_name[index_point], "w+t") # 무조건 처음 시작할 때 지우고 다시만든다.

    while True:

        resp = requests.get(base_url, params = params)

        soup = BeautifulSoup(resp.text, "lxml")

        news_list_tags = soup.select('#container > div > div.w_news_list.type_issue > ul > li > a.news')



        if(len(news_list_tags) == 0): # 더이상 탐색할 페이지가 없을때

            if(index_point + 1 == 7): # section 및 page 다 탐색 -> 종료

                f.close()
                break

            else:

                f.close()

                index_point += 1

                f = open(txt_file_name[index_point], "w+t")

                params['sectionType'] = index_list[index_point]

                params['pageIdx'] = 1

                continue

            break

        for i in news_list_tags: # 파일을 쓴다.

            dt_tags = i.find_all('strong')
            title_tag = dt_tags[0].text
            f.write(title_tag)
            f.write("\n")



        params['pageIdx'] += 1


    f.close()

# JTBC

In [2]:
def jtbc_news_list(day):
    
    # 카테고리 'scode'를 dict로 정의.
    menu_list = {
        '속보' : 0,
        '정치' : 10,
        '경제' : 20,
        '사회' : 30,
        '국제' : 40,
        '문화' : 50,
        '연예' : 60,
        '스포츠' : 70,
        '날씨': 80
    }
    
    base_url ='http://news.jtbc.joins.com/section/list.aspx?'
    params = {
        'scode' : 0,
        'pdate' : day,
        'pgi' : 0
    }
    
    # text 파일 통합 위해서 이름명 설정. 날씨 카테고리 문화에 합침.
    scode_list = {
        10 : 'a',
        20 : 'b',
        30 : 'c',
        40 : 'd',
        50 : 'e',
        60 : 'f',
        70 : 'g',
        80 : 'e'
    }
    
    for k in range(8):
        news_list = []
        params['scode'] +=10 # 카테고리 넘기기
        params['pgi'] = 0
        
        while True:
            params['pgi'] +=1 # 페이지 넘기기

            resp = requests.get(base_url, params=params)
            soup = BeautifulSoup(resp.text, 'lxml')

            news_tags = soup.find('ul', id='section_list')
            news_list_tags = news_tags.find_all('dt', class_='title_cr') # 뉴스 타이틀 tag


            if len(news_list_tags)==0: # 마지막 페이지에서 더 넘어가면 while문 탈출.
                break

            for tags in news_list_tags:
                news_title = tags.text

                news_list.append(
                    news_title.strip()
                )
        
    
        file_name = scode_list[params['scode']]+'2.txt' # 카테고리별 params 숫자를 받아 파일명 key값으로 입력 -> 알파벳 출력
        
        
        # 날씨 카테고리 내용 문화영역에 추가하기. 나머지는 그냥 파일 생성.
        if params['scode'] == 80:
            file1 = open(file_name, 'at')
        else:
            file1 = open(file_name, 'w')
        
        for i in news_list:
            file1.write(i+"\n")
            
        file1.close()

# YTN

In [3]:
# 오늘 날짜 
import re
import datetime

def make_f(newlist):
    # 파일명
    menu_list=['a3.txt','c3.txt','b3.txt',
          'd3.txt','e3.txt','g3.txt']
    for idx, i in enumerate(menu_list):
        f = open(i,'w')
        for j in newlist[idx]:
            msg = j
            f.write(j+"\n")
        f.close()
        #main()    

In [4]:
#스포츠 메뉴의 구조가 달라 따로 구함
def sport(Sports_list, day):
    for i in range(3):
        params={
            'page':i+1,
            's_mcd':'0107'
        }
        resp = requests.get('https://www.ytn.co.kr/photo/photo_list.php',params=params)
        soup = BeautifulSoup(resp.text, 'lxml')
        #리스트
        sec_tag = soup.find("div", id="ytn_list_v2014")
        dl_tag=sec_tag.find_all("dl",class_='photo_list')
        
        for j in dl_tag:
            # 날짜
            date_tag = j.find('dd',class_='date')
            date_d = date_tag.text
            date = date_d.split(" ")
            date = date[0].replace("[","")


            if day == date:
                dt_tag =j.find_all('dt')
                a_tag = dt_tag[0].find("a")
                title = a_tag.text
                
                
                if title in Sports_list:
                    continue
                    
                Sports_list.append(title)
                
      

In [5]:
def get_ytn():
    

    now = datetime.datetime.now() - datetime.timedelta(days=1)
    day = now.strftime("%Y-%m-%d")
    print(day)



    # YTN사이트
    url = 'https://www.ytn.co.kr/news/news_list_0101.html'

    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    menu = soup.find("ul",id='sub_1')
    li_tag = menu.find_all('li',recursive=False)

    a=[]
    politics_list=[]
    economy_list=[]
    social_list=[]
    domestic_list=[]
    International_list=[]
    Science_list=[]
    Sports_list=[]
    culture_list =[]


    # 카테고리마다 뉴스 제목 리스트
    newlist = [politics_list,economy_list,social_list,domestic_list,
               International_list,Science_list,culture_list]


    #3,4,5,6,7,8,9,10,11
    url2 = 'https://www.ytn.co.kr/news/news_list.php?'
    for idx in range(3,11,1):
        #메뉴이동
        a_tag =li_tag[idx].find('a')
        move = a_tag.get('href')
        a = re.findall('\d+',move)

        if idx == 10:
            sport(Sports_list, day)
            newlist.remove(domestic_list)
            newlist.remove(Science_list)
            newlist.append(Sports_list)
            make_f(newlist)
            break


        for i in range(3):
            params={
                'page':i+1,
                's_mcd':a[0]
            }
            resp = requests.get(url2,params=params)
            soup = BeautifulSoup(resp.text, 'lxml')

            #리스트
            sec_tag = soup.find("div", id="ytn_list_v2014")

            span_tag=sec_tag.find_all("dl", class_="news_list_v2014")

            for j in span_tag:
                # 날짜
                date_tag = j.find('dd',class_='date')
                date_d = date_tag.text
                date = date_d.split(" ")
                date = date[0].replace("[","")


                if day == date:
                    dt_tag =j.find_all('dt')
                    a_tag = dt_tag[0].find("a")
                    title = a_tag.text


                    if title in newlist[idx-3]:
                        continue
                    if idx == 6 or idx == 8:
                        newlist[6].append(title)

                    else:    
                        newlist[idx-3].append(title)



# MBN

In [6]:
############################################################
##########프로젝트 MBN 홈페이지 긁어오는 함수 코드##########
############################################################

import requests
import sys
from bs4 import BeautifulSoup
import json


def MBN_News_Function():
    
    count=0
    
    reresult_list=[]
    while count<7:
        
        if count==0:
            category_s='politics'
            MBN_url='https://www.mbn.co.kr/news/politics/'
        
        if count==1:
            category_s='economy'
            MBN_url='https://www.mbn.co.kr/news/economy/'

        elif count==2:
            category_s='society'
            MBN_url='https://www.mbn.co.kr/news/society/'

        elif count==3:
            category_s='world'
            MBN_url='https://www.mbn.co.kr/news/world/'

        elif count==4:
            category_s='culture'
            MBN_url='https://www.mbn.co.kr/news/culture/'

        elif count==5:
            category_s='entertain'
            MBN_url='https://www.mbn.co.kr/news/entertain/'        

        elif count==6:
            category_s='sports'
            MBN_url='https://www.mbn.co.kr/news/sports/'

        result_list=[]
        for i in range(1,8): #페이지번호
            params={
                'page':i,
                'vod':None,
                'category':category_s
            }
            resp=requests.get(MBN_url, params=params)

            resp.encoding='eur-kr'
            soup=BeautifulSoup(resp.text, "lxml")

            MBN_news_list=soup.select('div.list_area > dl.article_list > dt.tit > a')
            MBN_news_list_date=soup.select('div.list_area > dl.article_list > dd.desc > span.date')

            news_list=[]
            for news,date in zip(MBN_news_list, MBN_news_list_date):

                news=news.text.strip() #제목 내 띄어쓰기 제거
                news=news.replace("\'" , " ") #제목 내 따옴표 제거
                news=news.replace("♥" , " ") #제목 내 따옴표 제거
                news=news.replace("[포토]"," ") #제목 내 [포토] 제거

                date=date.text[0:10].replace("-","")

                if date == '20200806': #날짜 수정하면 됨
                    news_list.append(news)
            result_list.extend(news_list)
        
        count+=1
        
        
        reresult_list.append(result_list)
    return reresult_list





############################################################
#########프로젝트 MBN 메모장으로 꺼내오는 함수 코드#########
############################################################

# a1, 2, 3, 4 -> 정치
# b1, 2, 3, 4 -> 경제
# c1, 2, 3, 4 -> 사회
# d1, 2, 3, 4 -> 국제
# e1, 2, 3, 4 -> 생활&문화, 문화 + mbn생활건강 +ytn전국,과학
# f1, 2, 3, 4 -> 연예
# g1, 2, 3, 4 -> 스포츠

def MBN():

    title_list=['a','b','c','d','e','f','g']
    #나중에 j를 e로 바꿔서 추가해야함!

    f_list=[]
    for title in title_list:
        f=open('{}4.txt'.format(title),'w')    
        f_list.append(f)


    MBN=MBN_News_Function()   # 2중리스트로 옴. [[1,2,3...] , [1,2,3...] , [1,2,3...]]


    for i, f in zip(MBN,f_list):
        for j in i:
            f.write(j)
            f.write('\n')
        f.close()

# 아시아경제

In [7]:
############################################################
##########프로젝트 MBN 홈페이지 긁어오는 함수 코드##########
############################################################

import requests
import sys
from bs4 import BeautifulSoup
import json
# a1, 2, 3, 4 -> 정치
# b1, 2, 3, 4 -> 경제
# c1, 2, 3, 4 -> 사회
# d1, 2, 3, 4 -> 국제
# e1, 2, 3, 4 -> 생활&문화, 문화 + mbn생활건강 +ytn전국,과학
# f1, 2, 3, 4 -> 연예
# g1, 2, 3, 4 -> 스포츠

def ASIA_News_Function():
    
    count=0
    
    reresult_list=[]
    while count<6:
        
        if count==0:
            category_s='politics'
            #ASIA_url='https://www.asiae.co.kr/list/politics-all/1'
            ASIA_url='https://www.asiae.co.kr/news/list.htm?sec=0100'
            
        if count==1:
            category_s='economy'
            ASIA_url='https://www.asiae.co.kr/news/list.htm?sec=0200'

        elif count==2:
            category_s='society'
            ASIA_url='https://www.asiae.co.kr/news/list.htm?sec=0700'

        elif count==3:
            category_s='world'
            ASIA_url='https://www.asiae.co.kr/news/list.htm?sec=0600'

        elif count==4:
            category_s='culture'
            ASIA_url='https://www.asiae.co.kr/news/list.htm?sec=0800'

        elif count==5:
            category_s='entertain'
            ASIA_url='https://www.asiae.co.kr/news/list.htm?sec=1000'

        result_list=[]
        for i in range(2,3): #페이지번호

            resp=requests.get(ASIA_url)
            
            resp.encoding='eur-kr'
            soup=BeautifulSoup(resp.text, "lxml")

            ASIA_news_list=soup.select('div.cont_listarea > div > div.listsm_type > h3.l_fsttit > a')
        
            
            news_list=[]
            for news in ASIA_news_list:
                
                #본문
                news=news.text.strip() #제목 내 띄어쓰기 제거

                news_list.append(news)
                    
                    
            result_list.extend(news_list)
        
        count+=1
        
        
        reresult_list.append(result_list)
    return reresult_list




############################################################
#########프로젝트 MBN 메모장으로 꺼내오는 함수 코드#########
############################################################

# a1, 2, 3, 4 -> 정치
# b1, 2, 3, 4 -> 경제
# c1, 2, 3, 4 -> 사회
# d1, 2, 3, 4 -> 국제
# e1, 2, 3, 4 -> 생활&문화, 문화 + mbn생활건강 +ytn전국,과학
# f1, 2, 3, 4 -> 연예
# g1, 2, 3, 4 -> 스포츠

def ASIA():

    title_list=['a','b','c','d','e','f']
    #나중에 j를 e로 바꿔서 추가해야함!

    f_list=[]
    for title in title_list:
        f=open('{}5.txt'.format(title),'w')    
        f_list.append(f)


    ASIA=ASIA_News_Function()

    
    for i, f in zip(ASIA, f_list):
        for j in i:
            f.write(j)
            f.write('\n')
        f.close()

# 메인

In [8]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
from collections import Counter
import numpy as np
import matplotlib
from IPython.display import set_matplotlib_formats
from PIL import Image
import os



get_sbs()
jtbc_news_list(20200806)
get_ytn()
MBN()
ASIA()

################################# 텍스트에 Data 저장된 상태

txt_save_list = ['a.txt', 'b.txt', 'c.txt', 'd.txt', 'e.txt', 'f.txt', 'g.txt']

txt_file_list = [
    'a1.txt', 'a2.txt', 'a3.txt', 'a4.txt', 'a5.txt',
    'b1.txt', 'b2.txt', 'b3.txt', 'b4.txt', 'b5.txt' ,
    'c1.txt', 'c2.txt', 'c3.txt', 'c4.txt',  'c5.txt' ,
    'd1.txt', 'd2.txt', 'd3.txt', 'd4.txt', 'd5.txt' ,
    'e1.txt', 'e2.txt', 'e3.txt', 'e4.txt', 'e5.txt' ,
    'f1.txt', 'f2.txt', 'f3.txt', 'f4.txt','f5.txt' ,
    'g1.txt', 'g2.txt', 'g3.txt', 'g4.txt','g5.txt' 
]

title_name_list=['정치','경제','사회','국제','생활&문화','연예','스포츠']

#txt_file_list = [ 'a1.txt', 'a2.txt', 'a3.txt', 'b1.txt', 'b2.txt', 'b3.txt', 'c1.txt', 'c2.txt', 'c3.txt']



category_index_point = 0


while category_index_point < 7: # 카테고리 탐색
    
    
    s = [] #sbs
    d = [] #jtbc
    e = [] #ytn
    g = [] #mbn
    l = [] #asia

    f = open(txt_file_list[category_index_point * 5] , 'r') # 0, 5 , 10 , 15,20,25,30
    s = f.read()
    f.close()
    
    f = open(txt_file_list[(category_index_point*5) + 1] , 'r') #1,6,11,16,21,26,31
    d = f.read()
    f.close()
    
    if((category_index_point * 5) + 2) == 27:
        pass
    else:
        f = open(txt_file_list[(category_index_point*5) + 2] , 'r') #2,7,12,17,22,27,32
        e = f.read()
        f.close()
    
    f = open(txt_file_list[(category_index_point*5) + 3] , 'r') #3,8,13,18,23,38,33
    g = f.read()
    f.close()
    
    if((category_index_point * 5) + 4) == 34:
        pass
    else:
        f = open(txt_file_list[(category_index_point*5) + 4] , 'r') #4,9,14,19,24,39,34
        l = f.read()
        f.close()
    
    
    if((category_index_point * 5) + 2) != 27 and ((category_index_point * 5) + 4) != 34:
        s = s + d + e + g + l
        
    elif ((category_index_point * 5) + 2) == 27:
        s = s + d + g + l
    elif ((category_index_point * 5) + 4) == 34:
        s = s + d + g + e
        
    f = open(txt_save_list[category_index_point], 'w+t')
    f.write(s)
    f.close()
    
    if os.path.exists('total.txt'):
        f1 = open('total.txt', 'a')
        f1.write(s)
        f1.close()
    else:
        f1 = open('total.txt', 'w')
        f1.write(s)
        f1.close()
    
    
    
    
    category_index_point += 1 # 다음 카테고리
    
category_index_point = 0

while category_index_point < 7:
    
    #### 순위를 찾아내기위한 문장 리스트###
    line_list=[]
    
    f = open(txt_save_list[category_index_point], 'r')
    lines = f.read()
    line_list = lines.split('\n')
    

    nlpy = Twitter()
    nouns = nlpy.nouns(lines)
    count = Counter(nouns)

    tag_count = []
    tags = []
    

    
    for n, c in count.most_common(100):
        dics = {'tag' : n, 'count' : c}
        if len(dics['tag']) >= 2 and len(tags) <= 49:
            tag_count.append(dics)
            tags.append(dics['tag'])
    
    #### 순위 3위까지의 리스트#####
    rank=[]
    
    rank.clear()
    print('---------------카테고리[{}]--------------------'.format(title_name_list[category_index_point]) )
    for idx,tag in enumerate(tag_count):    
     #### 순위 3위까지의 리스트 뽑아내기 && 기사 뽑아내기#####
        if(idx == 0 or idx == 1 or idx == 2 ):
            rank.append(tag['tag'])   
        print(" {:<14}".format(tag['tag']), end='\t')
        print("{}".format(tag['count']))   
        
            
            
    print("\n ********** 순위 ************* \n")  
    print('\t 1순위:{}'.format(rank[0]))
    print('\t 2순위:{}'.format(rank[1]))
    print('\t 3순위:{}'.format(rank[2]))  
    
    rank_li=[]
    rank_li.clear() 
    
    temp=0
    for line_li in line_list:
        if rank[0] in line_li:
            temp+=1
            if temp == 2:
                break
            rank_li.append(line_li+'\n')

    temp=0
    for line_li in line_list:
        if rank[1] in line_li:
            temp+=1
            if temp == 2:
                break
            rank_li.append(line_li+'\n')
    temp=0
    for line_li in line_list:
        if rank[2] in line_li:
            temp+=1
            if temp == 2:
                break
            rank_li.append(line_li+'\n')  
                
    print("\n ********** 기사 ************* \n")
    print(rank_li[0])
    print(rank_li[1])
    print(rank_li[2])


                     
        
    new_list = []
    
    for tag in tag_count:
        a = tag['tag']
        b = tag['count']
        c = (a, b)
        new_list.append(c)
        
        
    alice_mask = np.array(Image.open("cloud.png"))
    font_path = 'C:/Windows/Fonts/malgun.ttf';
    wc = WordCloud(font_path = font_path, background_color = 'white', 
        width=800, height=600, mask = alice_mask)
    cloud = wc.generate_from_frequencies(dict(new_list))
    plt.figure(figsize=(10,8), dpi=200)
    plt.axis('off')
    plt.imshow(cloud)
    
    category_index_point += 1
    
    
    print('\n' * 10)
    



f3 = open('total.txt', 'r')

lines = f3.readline()
news_topic_count = 1

line_list.clear()
while lines:
    
    lines = f3.readline()
    news_topic_count += 1


    
print('전체 기사 수 :{}'.format(news_topic_count))

f3.close()
    

f = open('total.txt', 'r')
lines = f.read()
#### 순위를 찾아내기위한 문장 리스트###
line_list = lines.split('\n')
f.close()

nlpy = Twitter()
nouns = nlpy.nouns(lines)
count = Counter(nouns)

tag_count = []
tags = []



for n, c in count.most_common(100):
    dics = {'tag' : n, 'count' : c}
    if len(dics['tag']) >= 2 and len(tags) <= 49:
        tag_count.append(dics)
        tags.append(dics['tag'])
        
rank=[]
    
rank.clear()
print('---------------카테고리[전체 ]--------------------' )
for idx,tag in enumerate(tag_count):
 #### 순위 3위까지의 리스트 뽑아내기 && 기사 뽑아내기#####
    if(idx == 0 or idx == 1 or idx == 2 ):
        rank.append(tag['tag'])
        
    print(" {:<14}".format(tag['tag']), end='\t')
    print("{}".format(tag['count']))   



print("\n ********** 순위 ************* \n")  
print('\t 1순위:{}'.format(rank[0]))
print('\t 2순위:{}'.format(rank[1]))
print('\t 3순위:{}'.format(rank[2]))  

rank_li.clear() 


temp=0
for line_li in line_list:
    if rank[0] in line_li:
        temp+=1
        if temp == 2:
            break
        rank_li.append(line_li+'\n')
        
temp=0
for line_li in line_list:
    if rank[1] in line_li:
        temp+=1
        if temp == 2:
            break
        rank_li.append(line_li+'\n')
temp=0
for line_li in line_list:
    if rank[2] in line_li:
        temp+=1
        if temp == 2:
            break
        rank_li.append(line_li+'\n')        
            
print("\n ********** 기사 *************")
print(rank_li[0])
print(rank_li[1])
print(rank_li[2])




new_list = []

for tag in tag_count:
    a = tag['tag']
    b = tag['count']
    c = (a, b)
    new_list.append(c)


alice_mask = np.array(Image.open("cloud.png"))
font_path = 'C:/Windows/Fonts/malgun.ttf';
wc = WordCloud(font_path = font_path, background_color = 'white', 
    width=800, height=600, mask = alice_mask)
cloud = wc.generate_from_frequencies(dict(new_list))
plt.figure(figsize=(10,8), dpi = 200)
plt.axis('off')
plt.imshow(cloud)

    
os.remove('total.txt')

2020-08-06


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


---------------카테고리[정치]--------------------
 지원            	25
 원피스           	24
 호정            	21
 방류            	19
 북한            	16
 통합            	15
 특별            	14
 정부            	13
 논란            	12
 민주당           	12
 코로나           	11
 유착            	11
 뉴스            	11
 국회            	10
 김정은           	9
 유감            	9
 부동산           	9
 농지            	8
 윤석열           	8
 주호영           	8
 봉쇄            	8
 지역            	8
 조원            	7
 통보            	7
 개성            	7
 주택            	7
 대동강           	7
 대통령           	7
 황강            	6
 긴급            	6
 여성            	6
 지시            	6
 사전            	6
 안철수           	6
 정무            	6
 수위            	6
 방문            	6
 재난            	6
 해임            	6
 일방            	5
 한상혁           	5
 부지            	5
 위반            	5
 총리            	5
 대북            	5
 공개            	5
 발언            	5
 수도            	5
 아침            	5
 이인영           	5

 ********** 순위 ************* 

	 1순위:지원
	